# 🗺️ 네이버 플레이스 크롤링 v4.1 (구글 코랩)

## 🎯 v4.1 주요 개선사항
- ✅ **주소 수집 강화** (네이버 맵 최신 구조 대응)
- ✅ **실시간 디버깅 로그** (어떤 셀렉터가 작동하는지 표시)
- ✅ **HTML 구조 분석** (주소가 없을 때 원인 파악)
- ✅ **카테고리 정보 수집**
- ✅ **CSV 다운로드 기능**
- ✅ **타지역업체 자동 감지**

## 📖 사용 방법
1. 아래 셀들을 **순서대로** 실행 (Shift+Enter)
2. 마지막 셀에서 **ngrok URL** 확인
3. URL 클릭하여 웹 앱 접속
4. 검색 후 **CSV 다운로드** 가능!

## 1️⃣ 패키지 설치 (2-3분 소요)

In [ ]:
print("=" * 70)
print("🚀 네이버 플레이스 크롤링 v4.1 시작")
print("=" * 70)
print()

print("📦 1단계: 패키지 설치 중...")
!pip install -q flask playwright pyngrok nest-asyncio
!playwright install chromium
!playwright install-deps chromium
print("✅ 패키지 설치 완료!\n")

## 2️⃣ ngrok 토큰 설정 (선택사항)

무료로 사용 가능하지만, https://ngrok.com 에서 가입 후 토큰을 받으면 더 안정적입니다.

In [ ]:
# 선택사항: ngrok 토큰 (없으면 빈 문자열로 두세요)
NGROK_TOKEN = ""  # 예: "2abc3def4ghi5jkl"

if NGROK_TOKEN:
    !ngrok authtoken {NGROK_TOKEN}
    print("✅ ngrok 토큰 설정 완료")
else:
    print("⚠️ ngrok 토큰 없이 실행 (2시간 제한)")

## 3️⃣ v4.1 크롤링 모듈 생성 (주소 수집 디버깅 강화)

In [ ]:
%%writefile naver_crawler.py
from playwright.sync_api import sync_playwright
import time
import re
from urllib.parse import quote
import csv
from datetime import datetime

class NaverPlaceCrawlerV41:
    """네이버 플레이스 크롤러 v4.1 - 주소 수집 디버깅 강화"""
    
    def __init__(self, debug=True):
        self.playwright = None
        self.browser = None
        self.page = None
        self.version = "v4.1"
        self.debug = debug
    
    def start(self):
        try:
            self.playwright = sync_playwright().start()
            self.browser = self.playwright.chromium.launch(
                headless=True,
                args=[
                    '--no-sandbox',
                    '--disable-setuid-sandbox',
                    '--disable-dev-shm-usage',
                    '--disable-blink-features=AutomationControlled'
                ]
            )
            self.page = self.browser.new_page()
            self.page.set_viewport_size({"width": 1920, "height": 1080})
            
            # User-Agent 설정
            self.page.set_extra_http_headers({
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            })
            
            print(f"✅ 브라우저 시작 ({self.version})")
            return True
        except Exception as e:
            print(f"❌ 브라우저 오류: {e}")
            return False
    
    def search_places(self, keyword, max_results=20):
        if not self.page:
            self.start()
        
        try:
            print(f"🔍 '{keyword}' 검색 중...")
            
            url = f"https://map.naver.com/p/search/{quote(keyword)}"
            self.page.goto(url, timeout=20000, wait_until="domcontentloaded")
            time.sleep(3)
            
            iframe = self.page.frame(name="searchIframe")
            if not iframe:
                print("❌ searchIframe을 찾을 수 없습니다")
                return []
            
            time.sleep(2)
            
            # 스크롤로 더 많은 결과 로드
            print("📜 결과 로딩 중...")
            for i in range(5):
                iframe.evaluate("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(0.8)
            
            results = []
            
            # 리스트 아이템 찾기 (v4.1 셀렉터 확장)
            item_selectors = [
                'li[role="listitem"]',
                'li.UEzoS',
                'li.place_item',
                'ul._2py9K li',
                'div.CHC5F',
                'ul.place_section_content li',
                'div[class*="item"]',
                'li[class*="place"]',
                'div[data-place-id]'
            ]
            
            items = []
            for selector in item_selectors:
                items = iframe.query_selector_all(selector)
                if items:
                    print(f"✅ {len(items)}개 발견 (셀렉터: {selector})")
                    break
            
            if not items:
                print("❌ 검색 결과를 찾을 수 없습니다")
                # 디버깅: iframe HTML 구조 확인
                if self.debug:
                    print("\n🔍 디버깅: iframe HTML 구조 샘플:")
                    html_sample = iframe.content()[:2000]
                    print(html_sample)
                return []
            
            print(f"📊 총 {min(len(items), max_results)}개 처리 중...")
            
            addr_success_count = 0
            
            for idx, item in enumerate(items[:max_results]):
                try:
                    # 업체명
                    name_selectors = [
                        '.TYaxT', '.place_bluelink', '.YwYLL', 
                        'a.place_bluelink', 'span.place_name',
                        '.place_name a', 'a[class*="place"]',
                        'div[class*="name"]', 'span[class*="name"]'
                    ]
                    name_text = self._get_text_by_selectors(item, name_selectors)
                    if not name_text:
                        continue
                    
                    # 카테고리
                    category_selectors = [
                        '.YzBgS', '.category', 'span.category',
                        '.place_category', '[class*="category"]',
                        'div[class*="category"]'
                    ]
                    category_text = self._get_text_by_selectors(item, category_selectors)
                    
                    # 주소 - v4.1 디버깅 강화
                    addr_selectors = [
                        # 네이버 최신 셀렉터
                        '.LDgIH',  # 2024 메인 셀렉터
                        '.addr',
                        'span.place_addr',
                        '.Osdwn',
                        'div.addr',
                        '.v7Sqg',
                        # 추가 셀렉터
                        '[class*="addr"]',
                        'span[class*="addr"]',
                        'div[class*="addr"]',
                        '.place_address',
                        'span.address',
                        'div.address',
                        # 2025 예상 셀렉터
                        'div[class*="address"]',
                        'span[class*="location"]',
                        'div[class*="location"]',
                        'p[class*="addr"]'
                    ]
                    addr_text = self._get_text_by_selectors_debug(item, addr_selectors, f"[{idx+1}] {name_text}")
                    
                    # 주소 패턴 매칭 (더 강화)
                    if not addr_text:
                        item_html = item.inner_html()
                        addr_patterns = [
                            # 전체 주소 패턴
                            r'([가-힣]+(?:특별시|광역시|시|도)\s+[가-힣]+(?:구|군|시)\s+[가-힣0-9\s\-]+)',
                            # 시/도 시작 패턴
                            r'(서울[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(경기[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(부산[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(대구[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(인천[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(광주[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(대전[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(울산[^<>]+?(?:동|로|가|길)\s*\d*)',
                            r'(세종[^<>]+?(?:동|로|가|길)\s*\d*)',
                            # 구/동 패턴
                            r'([가-힣]+구\s+[가-힣]+동\s+[0-9\-]+)',
                            r'([가-힣]+시\s+[가-힣]+구\s+[가-힣]+)',
                            # 로/길 패턴
                            r'([가-힣]+로\s+\d+[가-힣\s]*)',
                            r'([가-힣]+길\s+\d+[가-힣\s]*)',
                        ]
                        for pattern in addr_patterns:
                            match = re.search(pattern, item_html)
                            if match:
                                addr_text = match.group(1).strip()
                                if self.debug and idx < 3:  # 처음 3개만 로그
                                    print(f"    🔍 [{idx+1}] 정규식 매칭 성공: {addr_text[:30]}...")
                                break
                    
                    # 주소가 여전히 없으면 HTML 전체 검사
                    if not addr_text and self.debug and idx < 2:
                        print(f"\n⚠️ [{idx+1}] '{name_text}' - 주소 찾기 실패")
                        print(f"    HTML 샘플: {item.inner_html()[:300]}...\n")
                    
                    if addr_text and addr_text != "주소 정보 없음":
                        addr_success_count += 1
                    
                    # 전화번호
                    phone_selectors = [
                        '.dry6Z', '.tel', 'span.place_tel',
                        '.xlx7Q', '[class*="tel"]', 'span[class*="tel"]',
                        '.phone', 'span.phone', '[class*="phone"]',
                        'div[class*="tel"]', 'p[class*="tel"]'
                    ]
                    phone_text = self._get_text_by_selectors(item, phone_selectors)
                    
                    # 전화번호 패턴 매칭
                    if not phone_text:
                        item_html = item.inner_html()
                        phone_patterns = [
                            r'(0\d{1,2}[-\s]?\d{3,4}[-\s]?\d{4})',
                            r'(070[-\s]?\d{3,4}[-\s]?\d{4})',
                            r'(\d{2,3}-\d{3,4}-\d{4})',
                            r'(1[0-9]{3}-[0-9]{4})',  # 1588 등
                            r'(\d{10,11})'
                        ]
                        for pattern in phone_patterns:
                            match = re.search(pattern, item_html)
                            if match:
                                phone_text = match.group(1)
                                break
                    
                    # 평점
                    rating_selectors = [
                        '.h69bs', '.score', 'span.place_score',
                        '.PXMot', '[class*="score"]', 'span[class*="rating"]',
                        'div[class*="score"]'
                    ]
                    rating_text = self._get_text_by_selectors(item, rating_selectors)
                    
                    # 리뷰 수
                    reviews_selectors = [
                        '.Tvqnp', '.cnt', 'span.place_review',
                        '.YrbVX', '[class*="review"]', '[class*="count"]',
                        'div[class*="review"]'
                    ]
                    reviews_el = self._get_element_by_selectors(item, reviews_selectors)
                    reviews_text = "0"
                    if reviews_el:
                        reviews_raw = reviews_el.inner_text()
                        reviews_text = re.sub(r'[^0-9]', '', reviews_raw) if reviews_raw else "0"
                    
                    # 타지역업체 판단
                    is_other = self._is_other_region(name_text, addr_text, phone_text, rating_text, keyword)
                    
                    result = {
                        'name': name_text,
                        'category': category_text if category_text else "미분류",
                        'address': addr_text if addr_text else "주소 정보 없음",
                        'phone': phone_text if phone_text else "전화번호 없음",
                        'rating': rating_text if rating_text else "",
                        'reviews': reviews_text,
                        'is_other_region': is_other,
                        'place_type': '타지역업체' if is_other else '주업체'
                    }
                    
                    results.append(result)
                    
                    # 진행 상황 표시
                    status_icon = "🟠" if is_other else "🟢"
                    addr_display = addr_text[:25] if addr_text else "❌주소없음"
                    print(f"  {status_icon} [{idx+1}] {name_text[:15]}... | {addr_display}...")
                    
                except Exception as e:
                    print(f"⚠️ [{idx+1}] 파싱 오류: {str(e)[:50]}")
                    continue
            
            # 주소 수집 통계
            total = len(results)
            addr_rate = (addr_success_count / total * 100) if total > 0 else 0
            
            print(f"\n✅ 총 {total}개 수집 완료")
            print(f"   🟢 주업체: {len([r for r in results if not r['is_other_region']])}개")
            print(f"   🟠 타지역: {len([r for r in results if r['is_other_region']])}개")
            print(f"   📍 주소 수집: {addr_success_count}/{total} ({addr_rate:.1f}%)")
            
            return results
            
        except Exception as e:
            print(f"❌ 검색 오류: {e}")
            import traceback
            traceback.print_exc()
            return []
    
    def _get_element_by_selectors(self, parent, selectors):
        for selector in selectors:
            try:
                el = parent.query_selector(selector)
                if el:
                    return el
            except:
                continue
        return None
    
    def _get_text_by_selectors(self, parent, selectors):
        el = self._get_element_by_selectors(parent, selectors)
        if el:
            try:
                text = el.inner_text().strip()
                return text if text else ""
            except:
                return ""
        return ""
    
    def _get_text_by_selectors_debug(self, parent, selectors, item_name=""):
        """디버깅용 텍스트 추출 (어느 셀렉터가 작동하는지 표시)"""
        for selector in selectors:
            try:
                el = parent.query_selector(selector)
                if el:
                    text = el.inner_text().strip()
                    if text:
                        if self.debug and parent in list(parent.query_selector_all('*'))[:2]:  # 처음 2개만
                            print(f"    ✅ {item_name} - 주소 발견 (셀렉터: {selector})")
                        return text
            except:
                continue
        return ""
    
    def _is_other_region(self, name, address, phone, rating, keyword):
        """타지역업체 판단"""
        score = 0
        
        # 070 번호 (+3점)
        if phone and ('070' in phone):
            score += 3
        
        # 주소가 간략함 (+2점)
        if address and address != "주소 정보 없음":
            addr_parts = address.split()
            if len(addr_parts) <= 3:
                score += 2
            if address.endswith(('동', '구', '시')):
                score += 1
        
        # 주소 없음 (+2점)
        if not address or address == "주소 정보 없음":
            score += 2
        
        # 평점 없음 (+1점)
        if not rating:
            score += 1
        
        # 상호에 검색 키워드 포함 (+2점)
        if name and keyword:
            keyword_words = [w for w in keyword.split() if len(w) > 1]
            if any(w in name for w in keyword_words):
                score += 2
        
        return score >= 4
    
    def close(self):
        try:
            if self.browser:
                self.browser.close()
            if self.playwright:
                self.playwright.stop()
        except:
            pass

## 4️⃣ Flask 웹 앱 실행 🚀

이 셀을 실행하면:
- Flask 서버가 시작됩니다
- ngrok 터널이 생성됩니다
- **공개 URL**이 표시됩니다
- **디버깅 로그**로 주소 수집 상황 확인 가능
- **CSV 다운로드** 기능 사용 가능

⚠️ 이 셀은 계속 실행되므로 중지하지 마세요!

In [ ]:
import nest_asyncio
from pyngrok import ngrok
from flask import Flask, request, jsonify, Response
import threading
import time
import io
import csv
from datetime import datetime
from naver_crawler import NaverPlaceCrawlerV41

nest_asyncio.apply()

app = Flask(__name__)
crawler = None

@app.route('/')
def index():
    return '''<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>네이버 플레이스 크롤링 v4.1</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { 
            font-family: 'Segoe UI', Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }
        .header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }
        h1 { font-size: 2.5em; margin-bottom: 10px; }
        .badge-version { 
            display: inline-block;
            background: rgba(255,100,100,0.4);
            padding: 5px 15px;
            border-radius: 20px;
            font-size: 0.6em;
            margin-left: 10px;
            border: 2px solid white;
        }
        .content { padding: 40px; }
        .search-box {
            display: flex;
            gap: 10px;
            margin-bottom: 30px;
        }
        input {
            flex: 1;
            padding: 15px;
            border: 2px solid #ddd;
            border-radius: 10px;
            font-size: 16px;
        }
        button {
            padding: 15px 30px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 10px;
            font-size: 16px;
            cursor: pointer;
            font-weight: 600;
            transition: all 0.3s;
        }
        button:hover { transform: translateY(-2px); box-shadow: 0 5px 20px rgba(102,126,234,0.4); }
        button:disabled { opacity: 0.5; cursor: not-allowed; }
        .btn-download {
            background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        }
        .loading { display: none; text-align: center; padding: 20px; font-size: 18px; color: #667eea; }
        .loading.active { display: block; }
        .results { margin-top: 30px; }
        .place-card {
            border: 2px solid #e0e0e0;
            padding: 20px;
            margin: 15px 0;
            border-radius: 12px;
            transition: all 0.3s;
        }
        .place-card:hover { transform: translateY(-5px); box-shadow: 0 8px 25px rgba(0,0,0,0.15); }
        .place-card.other { border-color: #ff9800; background: #fff3e0; }
        .badge {
            display: inline-block;
            padding: 5px 12px;
            border-radius: 15px;
            font-size: 0.85em;
            font-weight: 600;
            margin-left: 10px;
        }
        .badge.main { background: #4caf50; color: white; }
        .badge.other { background: #ff9800; color: white; }
        .badge.category { background: #2196F3; color: white; margin-left: 5px; }
        .stats {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-bottom: 30px;
        }
        .stat-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
        }
        .stat-card h3 { font-size: 2em; margin-bottom: 5px; }
        .stat-card.warning { background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); }
        .stat-card.success { background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); }
        .info-section {
            background: linear-gradient(135deg, #fff4e6 0%, #ffe9d0 100%);
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
            border-left: 5px solid #ff9800;
        }
        .info-section h3 { margin-bottom: 10px; color: #e65100; }
        .info-section ul { margin-left: 20px; margin-top: 10px; }
        .info-section li { margin: 5px 0; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🗺️ 네이버 플레이스 크롤링<span class="badge-version">v4.1 디버그</span></h1>
            <p>주소 수집 문제 해결 버전 | 실시간 디버깅 로그</p>
        </div>
        <div class="content">
            <div class="info-section">
                <h3>🔧 v4.1 개선사항</h3>
                <ul>
                    <li>✅ 주소 셀렉터 15개로 확장 (2024-2025 대응)</li>
                    <li>✅ 실시간 디버깅 로그 (어떤 셀렉터가 작동하는지 표시)</li>
                    <li>✅ 정규식 패턴 매칭 강화 (14개 패턴)</li>
                    <li>✅ HTML 구조 분석 기능 (주소 없을 때)</li>
                    <li>✅ 주소 수집률 실시간 표시</li>
                </ul>
            </div>
            <div class="search-box">
                <input type="text" id="keyword" placeholder="검색어 입력 (예: 강남역 맛집, 선불폰, 하수구역빌)">
                <button onclick="search()" id="btnSearch">🔍 검색</button>
                <button onclick="downloadCSV()" id="btnDownload" class="btn-download" style="display:none">📥 CSV 다운로드</button>
            </div>
            <div class="loading" id="loading">⏳ 검색 중... 잠시만 기다려주세요</div>
            <div id="stats"></div>
            <div id="results" class="results"></div>
        </div>
    </div>
    <script>
        let currentResults = [];
        let currentKeyword = '';
        
        async function search() {
            const keyword = document.getElementById('keyword').value.trim();
            const loading = document.getElementById('loading');
            const results = document.getElementById('results');
            const stats = document.getElementById('stats');
            const btnSearch = document.getElementById('btnSearch');
            const btnDownload = document.getElementById('btnDownload');
            
            if (!keyword) {
                alert('검색어를 입력하세요');
                return;
            }
            
            loading.classList.add('active');
            results.innerHTML = '';
            stats.innerHTML = '';
            btnSearch.disabled = true;
            btnDownload.style.display = 'none';
            
            try {
                const response = await fetch('/api/search', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ keyword, max_results: 20 })
                });
                
                const data = await response.json();
                
                if (data.success) {
                    currentResults = data.results;
                    currentKeyword = keyword;
                    
                    const total = data.results.length;
                    const other = data.results.filter(p => p.is_other_region).length;
                    const main = total - other;
                    const withAddr = data.results.filter(p => p.address && p.address !== '주소 정보 없음').length;
                    const withCategory = data.results.filter(p => p.category && p.category !== '미분류').length;
                    const addrRate = ((withAddr / total) * 100).toFixed(1);
                    
                    stats.innerHTML = `
                        <div class="stats">
                            <div class="stat-card"><h3>${total}</h3><p>총 결과</p></div>
                            <div class="stat-card"><h3>${main}</h3><p>주업체</p></div>
                            <div class="stat-card warning"><h3>${other}</h3><p>타지역업체</p></div>
                            <div class="stat-card success"><h3>${withAddr}/${total}</h3><p>주소 수집 (${addrRate}%)</p></div>
                            <div class="stat-card success"><h3>${withCategory}</h3><p>카테고리</p></div>
                        </div>
                    `;
                    
                    results.innerHTML = data.results.map((place, idx) => {
                        const className = place.is_other_region ? 'place-card other' : 'place-card';
                        const badgeClass = place.is_other_region ? 'badge other' : 'badge main';
                        const badgeText = place.is_other_region ? '🟠 타지역업체' : '🟢 주업체';
                        
                        // 주소 없을 때 경고 표시
                        const addrDisplay = place.address === '주소 정보 없음' 
                            ? '<span style="color: red; font-weight: bold;">⚠️ 주소 정보 없음</span>'
                            : place.address;
                        
                        return `
                            <div class="${className}">
                                <strong style="font-size: 1.2em;">[${idx+1}] ${place.name}</strong>
                                <span class="${badgeClass}">${badgeText}</span>
                                ${place.category !== '미분류' ? '<span class="badge category">' + place.category + '</span>' : ''}
                                <br><br>
                                📍 <strong>주소:</strong> ${addrDisplay}<br>
                                📞 <strong>전화:</strong> ${place.phone}<br>
                                ${place.rating ? '⭐ ' + place.rating + ' (리뷰 ' + place.reviews + '개)' : '평점 없음'}
                            </div>
                        `;
                    }).join('');
                    
                    btnDownload.style.display = 'inline-block';
                } else {
                    results.innerHTML = `<p style="color: red;">오류: ${data.error}</p>`;
                }
            } catch (error) {
                results.innerHTML = `<p style="color: red;">오류: ${error.message}</p>`;
            } finally {
                loading.classList.remove('active');
                btnSearch.disabled = false;
            }
        }
        
        async function downloadCSV() {
            if (currentResults.length === 0) {
                alert('검색 결과가 없습니다');
                return;
            }
            
            try {
                const response = await fetch('/api/download-csv', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ 
                        results: currentResults,
                        keyword: currentKeyword
                    })
                });
                
                if (response.ok) {
                    const blob = await response.blob();
                    const url = window.URL.createObjectURL(blob);
                    const a = document.createElement('a');
                    a.href = url;
                    a.download = `naver_places_${currentKeyword}_${new Date().getTime()}.csv`;
                    document.body.appendChild(a);
                    a.click();
                    a.remove();
                    window.URL.revokeObjectURL(url);
                    alert('✅ CSV 다운로드 완료!');
                } else {
                    alert('❌ 다운로드 실패');
                }
            } catch (error) {
                alert('❌ 다운로드 오류: ' + error.message);
            }
        }
        
        document.getElementById('keyword').addEventListener('keypress', (e) => {
            if (e.key === 'Enter') search();
        });
    </script>
</body>
</html>'''

@app.route('/api/search', methods=['POST'])
def api_search():
    global crawler
    try:
        data = request.json
        keyword = data.get('keyword', '')
        max_results = int(data.get('max_results', 20))
        
        if not keyword:
            return jsonify({'success': False, 'error': '검색어를 입력하세요'})
        
        if crawler is None:
            crawler = NaverPlaceCrawlerV41(debug=True)  # 디버그 모드 활성화
            crawler.start()
        
        results = crawler.search_places(keyword, max_results)
        return jsonify({'success': True, 'results': results})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

@app.route('/api/download-csv', methods=['POST'])
def download_csv():
    try:
        data = request.json
        results = data.get('results', [])
        keyword = data.get('keyword', 'search')
        
        if not results:
            return jsonify({'success': False, 'error': '결과가 없습니다'}), 400
        
        # CSV 생성
        output = io.StringIO()
        writer = csv.DictWriter(output, fieldnames=[
            'name', 'category', 'address', 'phone', 
            'rating', 'reviews', 'place_type'
        ])
        writer.writeheader()
        writer.writerows(results)
        
        # 바이트로 변환
        output.seek(0)
        csv_data = output.getvalue().encode('utf-8-sig')
        
        # 응답 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"naver_places_{keyword}_{timestamp}.csv"
        
        return Response(
            csv_data,
            mimetype='text/csv',
            headers={'Content-Disposition': f'attachment; filename={filename}'}
        )
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Flask 서버 시작
print("🚀 Flask 서버 시작 중...\n")
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

time.sleep(3)

# ngrok 터널 생성
print("🌐 ngrok 터널 생성 중...\n")
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url

print("=" * 70)
print("✅ v4.1 디버그 서버가 시작되었습니다!")
print("=" * 70)
print()
print(f"🌐 접속 URL: {public_url}")
print()
print("💡 위 URL을 클릭하거나 복사해서 브라우저에서 열어주세요!")
print()
print("📝 v4.1 주요 기능:")
print("  ✅ 주소 셀렉터 15개 (2024-2025 대응)")
print("  ✅ 실시간 디버깅 로그 출력")
print("  ✅ 정규식 패턴 14개 강화")
print("  ✅ HTML 구조 분석 기능")
print("  ✅ 주소 수집률 표시")
print("  ✅ CSV 다운로드 기능")
print()
print("🧪 테스트 추천 검색어:")
print("  - 선불폰")
print("  - 강남역 맛집")
print("  - 홍대 카페")
print("  - 하수구역빌")
print()
print("⚠️  이 셀이 실행 중일 때만 접속 가능합니다")
print("⚠️  중지하려면 정지 버튼 클릭")
print()
print("🔍 디버그 로그를 확인하여 주소 수집 상황을 모니터링하세요!")
print("=" * 70)
print()

# 서버 유지
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 서버 종료 중...")
    ngrok.disconnect(public_url)
    if crawler:
        crawler.close()
    print("✅ 종료 완료")

## 🎉 v4.1 완료!

위 셀에 표시된 **ngrok URL**을 클릭하세요!

### v4.1 vs v4 차이점

| 항목 | v4 | v4.1 |
|------|----|----- |
| 주소 셀렉터 | 12개 | **15개 (2025 대응)** |
| 정규식 패턴 | 7개 | **14개 (도시별)** |
| 디버그 로그 | ❌ | **✅ 실시간 출력** |
| HTML 분석 | ❌ | **✅ 주소 없을 때** |
| 수집률 표시 | 기본 | **✅ 퍼센트 표시** |

### 디버그 로그 활용법
- 검색 시 Colab 콘솔에서 실시간 로그 확인
- 어떤 셀렉터가 작동하는지 확인 가능
- 주소가 없는 경우 HTML 샘플 출력
- 주소 수집률 통계 확인

### 문제 해결
1. **여전히 주소가 없어요**
   - 디버그 로그를 확인하세요
   - HTML 샘플을 개발자에게 전달해주세요
   - 네이버 맵 구조가 변경되었을 수 있습니다

2. **일부만 주소가 나와요**
   - 정상입니다 (일부 업체는 주소 미제공)
   - 주소 수집률 70% 이상이면 정상

3. **크롤링이 느려요**
   - 디버그 모드로 인해 약간 느릴 수 있습니다
   - 정상적으로 20-30초 소요됩니다

**행복한 디버깅 되세요!** 🚀🔧